In [2]:
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import StandardScaler


# Set random seed for reproducibility
np.random.seed(42)

# Simulation parameters
num_providers = 3
num_regions = 3
min_dcs_per_provider = 2
max_dcs_per_provider = 5
num_vms_per_dc = 8
vm_processing_capability = 1500  # in MIPS
vm_num_cpus = 2
vm_ram = 4  # in Gb
vm_storage_capacity = 8  # in Gb
num_data = 200  # Number of data
data_size_range = (300, 1000)  # Data size range in Mb
task_size_range = (200, 1000)  # Task size range in MI
Re = 0.7  # Provider revenues per task execution ($)
C_penalty = 0.0025  # Penalty per violation ($)
num_dc_list = [6, 9, 12, 15]

inter_region_bw_capacity = 500  # in Mb/s
inter_region_bw_delay = 150  # in ms

intra_region_bw_capacity = 1000  # in Mb/s
intra_region_bw_delay = 50  # in ms

intra_dc_bw_capacity = 8000  # in Mb/s
intra_dc_bw_delay = 10  # in ms

# Pricing information
operating_cost = {
    'Provider 1': {
        'US': [0.020, 0.006, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'EU': [0.025, 0.006, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'AS': [0.027, 0.0066, 0.001, 0.0015, 0.002, 0.004, 0.008],
    },
    'Provider 2': {
        'US': [0.020, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'EU': [0.018, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'AS': [0.020, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
    },
    'Provider 3': {
        'US': [0.0095, 0.00120, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'EU': [0.0090, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'AS': [0.0080, 0.0090, 0.001, 0.0015, 0.002, 0.004, 0.008],
    },
}

# Simulate data for each provider and region
for num_dcs in num_dc_list:
    provider_data = []  # Reset provider data for each iteration
    for provider_id in range(1, num_providers + 1):
        for region in ['US', 'EU', 'AS']:
            for dc_id in range(1, num_dcs + 1):                
                for data_id in range(1, num_data + 1):
                    data_size = np.random.uniform(data_size_range[0], data_size_range[1])
                    task_size = np.random.uniform(task_size_range[0], task_size_range[1])
                    
                    base_response_time = 180
                    response_time_variation = np.random.normal(loc=0, scale=10)
                    response_time = max(0, base_response_time + response_time_variation)
                    
                    # Include bandwidth and delay information in the data
                    vm_data = {
                        'provider_id': provider_id,
                        'region': region,
                        'dc_id': dc_id,
                        'num_vms': num_vms_per_dc,
                        'vm_processing_capability': vm_processing_capability,
                        'vm_num_cpus': vm_num_cpus,
                        'vm_ram': vm_ram,
                        'vm_storage_capacity': vm_storage_capacity,
                        'cpu_cost': operating_cost[f'Provider {provider_id}'][region][0],
                        'storage_cost': operating_cost[f'Provider {provider_id}'][region][1],
                        'intra_dc_bw_cost': operating_cost[f'Provider {provider_id}'][region][2],
                        'inter_region_bw_cost': operating_cost[f'Provider {provider_id}'][region][3],
                        'intra_region_bw_cost': operating_cost[f'Provider {provider_id}'][region][4],
                        'response_time_slo': response_time,
                        'availability_slo': 0.95,
                        'task_count': np.random.choice([1000, 2000, 3000, 5000, 7000, 10000]),
                        'data_size': data_size,
                        'task_size': task_size,
                        'inter_region_bw_capacity': inter_region_bw_capacity,
                        'inter_region_bw_delay': inter_region_bw_delay,
                        'intra_region_bw_capacity': intra_region_bw_capacity,
                        'intra_region_bw_delay': intra_region_bw_delay,
                        'intra_dc_bw_capacity': intra_dc_bw_capacity,
                        'intra_dc_bw_delay': intra_dc_bw_delay,
                    }
                    provider_data.append(vm_data)
relevant_features = [
    'response_time_slo', 'availability_slo', 'cpu_cost', 'storage_cost',
    'intra_dc_bw_cost', 'inter_region_bw_cost', 'intra_region_bw_cost',
    'task_count', 'data_size', 'task_size'
]

cluster_data = np.array([[vm.get(feature, 0) for feature in relevant_features] for vm in provider_data])

# Standardize the data
scaler = StandardScaler()
cluster_data_scaled = scaler.fit_transform(cluster_data)

# Step 2: Spectral Clustering
spectral = SpectralClustering(n_clusters=3, affinity='nearest_neighbors', n_neighbors=5)
cluster_labels = spectral.fit_predict(cluster_data_scaled)

# Cluster Analysis
cluster_density = np.bincount(cluster_labels)
most_critical_clusters = np.argsort(cluster_density)[-2:]
critical_data_indices = np.hstack([np.where(cluster_labels == cluster)[0] for cluster in most_critical_clusters])
critical_data = cluster_data[critical_data_indices]


# Placeholder for fuzzy logic categorization functions
def categorize_data_transfer_time(response_time):
    # Placeholder for categorizing data transfer time
    return "high" if response_time > 200 else "medium" if response_time > 150 else "low"

def categorize_vm_load(task_count):
    # Placeholder for categorizing VM load
    return "high" if task_count > 7000 else "medium" if task_count > 3000 else "low"

def categorize_data_availability(availability):
    # Placeholder for categorizing data availability
    return "Respected" if availability > 0.95 else "Non-respected"

def categorize_provider_profit(cpu_price, storage_price, total_revenue):
    # Placeholder for categorizing provider profit
    total_cost = cpu_price + storage_price  # Simplified cost calculation
    return "Profit" if total_revenue - total_cost > 0 else "Non-Profit"

for data in critical_data:
    data['data_transfer_time_category'] = categorize_data_transfer_time(data['response_time'])
    data['vm_load_category'] = categorize_vm_load(data['task_count'])
    data['data_availability_category'] = categorize_data_availability(data['availability'])
    # Assuming 'total_revenue' is a placeholder for revenue generated from this resource
    data['provider_profit_category'] = categorize_provider_profit(data['cpu_price'], data['storage_price'], total_revenue)

categorized_data = [{k: data[k] for k in ['data_transfer_time_category', 'vm_load_category', 'data_availability_category', 'provider_profit_category']} for data in critical_data]
print(categorized_data[:5]) 

c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices